#Running Mask-RCNN

In [ ]:
import os
#%tensorflow_version 1.x

In [ ]:
os.getcwd()

In [ ]:
#os.listdir("../../../../inputs/")
#os.chdir("samples/construction")

### Training

In [ ]:
!pip install keras==2.1.5
!pip install tensorflow-gpu==1.15.2
!pip install tensorflow==1.15.2

In [ ]:
#!python construction.py train --dataset=/../../../../inputs/ --weights=coco

In [ ]:
os.listdir()

## Inference

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the model
#ROOT_DIR = os.path.abspath("../../")
ROOT_DIR = ""

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
#from mrcnn.visualize import display_images
import mrcnn.model as modellib
#from mrcnn.model import log
from samples.construction import construction

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
CONSTRUCTION_WEIGHTS_PATH = "logs/mask_rcnn_construction_0030.h5"  # TODO: update this path

In [ ]:
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

In [ ]:
config = construction.ConstructionConfig()
CONSTRUCTION_DIR = os.path.join(ROOT_DIR, "../../inputs/datasets/")

# New Section

In [ ]:
from mrcnn.config import Config

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 100
    BATCH_SIZE = 100

config = InferenceConfig()
config.display()

In [ ]:
# Create model in inference mode
import tensorflow as tf
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [ ]:
# Set path to construction weights file
weights_path = "logs/mask_rcnn_construction_0030.h5"

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
from Mask_RCNN.utils import load_labels_from_folder, reformat_label, is_polygon, reshape_poly_coordinates, filter_on_image_id

In [ ]:
dataset_test = utils.Dataset()
TEST_DIR = "../../inputs/Test"
source = "construction"

In [ ]:
# Add classes
dataset_test.add_class("construction", 1, "Vertical_formwork")
dataset_test.add_class("construction", 2, "Concrete_pump_hose")

# Add images
image_paths = [os.path.join(TEST_DIR, image_name) for image_name in sorted(os.listdir(TEST_DIR))]
image_paths.pop(image_paths.index('../../inputs/Test/.ipynb_checkpoints'))
i = 0
for image_path in image_paths: 
    img = plt.imread(image_path)
    width = img.shape[0]
    height = img.shape[1]
    dataset_test.add_image(source, i, image_path, width=width,height=height)
    i += 1

dataset_test.prepare()

In [ ]:
print("Images: {}\nClasses: {}".format(len(dataset_test.image_ids), dataset_test.class_names))

In [ ]:
images = []
for image_id in dataset_test.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_test, config, image_id, use_mini_mask=False)
    info = dataset_test.image_info[image_id]
    images.append(image)

In [ ]:
result = model.detect(images, verbose=1)

In [ ]:
result

In [ ]:
import json
def jsonify_array(array):
    d = {}
    ## Merge dictionaries into one
    for k in results[0].keys():
        d[k] = tuple(d[k] for d in results)
    # Prepare output results
    out = dict.fromkeys(d.keys())
    for key in out.keys():
        # Cast numpy arrays to lists
        temp_list = [a.tolist() for a in d[key]]
        final_list = [val for sublist in temp_list for val in sublist]
        out[key] = final_list
    return out

In [ ]:
#out = jsonify_results(results)

In [ ]:
#with open('results.json', 'w') as f:
#    json.dump(out, f)